# Анализ текстов судебных решений

ВКР "Алименты в России: медиапроект, основанный на данных"

Морозова Николь, НИУ ВШЭ, магистерская программа Журналистика данных

Руководитель: м.с.н., Груздев И.А.

### Выборка
Предмет: Статья 157. Неуплата средств на содержание детей или нетрудоспособных родителей (Уголовный Кодекс)

Хронологические рамки: 1 января 2018 – 31 декабря 2018

Источник: https://sudact.ru/

In [104]:
# загружаем спаршенный набор текстов
import json

with open('/Users/Nikol/Desktop/ВКР/sudact/uk_texts_2018.json') as f:
    uk_all = json.load(f)

### Исключаем лишнее, оставляем только приговоры

In [107]:
only_prigs = []
for text in uk_all:
    text_type = text.split(' № ')[0]
    if 'приг' in text_type.lower():
        only_prigs.append(text)
print('Первоначальная выборка: {} текстов. Новая: {} текстов.'.format(len(uk_all),len(only_prigs)))

Первоначальная выборка: 1513 текстов. Новая: 1045 текстов.


In [108]:
def find_word(lines, searched_line):
    for iline in range(len(lines)):
        line = lines[iline].replace(' ', '').lower()
        if searched_line in line:
            return iline
    return False

def get_region(lines):
    region = lines[1].split('(')[1].split(')')[0]
    return region

def get_sex(lines):
    for iline in range(len(lines)):
        if ('обвиняемого' in lines[iline]):
            return 'муж'
        if ('обвиняемой' in lines[iline]):
            return 'жен'

def get_crime(lines):
    s2 = find_word(lines,'установил')

    for iline in range(len(lines)):
        if 'предусмотрен' in lines[iline].lower():
            crime_mention = iline
            break

    try:
        crime_line = ' '.join(lines[crime_mention:s2]).split('предусмотрен')[1].split(' ')[1:]
        crime = ' '.join(crime_line).split('У')[0]
        return crime

    except:
        return

def get_record(lines):
    s2 = find_word(lines,'установил')

    crime_rec = 'no'
    crime_rec_info = 'no'

    for iline in range(len(lines[:s2])):
        if 'ранее судим' in lines[iline].lower():
            crime_rec = 'yes'
            break

    return crime_rec

def get_record_crimes(lines):
    s2 = find_word(lines,'установил')

    crime_rec = 'no'
    crime_rec_info = 'no'

    for iline in range(len(lines[:s2])):
        if 'ранее судим' in lines[iline]:
            crime_rec = 'yes'
            crime_rec_info = ' '.join(lines[iline:s2])
            break

    crime_rec_info_lst = []
    crime_rec_info_lst = crime_rec_info.split('ст.')
    rec_crimes_lst = []

    for line in crime_rec_info_lst:
        if any([str(i) in line for i in range(10)]) and ('У' in line) and ('К'in line):
            rec_crime = str(line).split('У')[0]
            rec_crimes_lst.append(rec_crime)
    if len(rec_crimes_lst) > 0:
        return rec_crimes_lst
    else:
        return 'no'

def compile_dict(text):
    lines = text.split('\n')
    case = {}
    case['region'] = get_region(lines)
    case['sex'] = get_sex(lines)
    case['crime'] = get_crime(lines)
    case['record'] = get_record(lines)
    case['record_crimes'] = get_record_crimes(lines)
    return case

In [109]:
details_uk = []
for text in only_prigs:
    details_uk.append(compile_dict(text))

In [110]:
# сохраняем результат в новом файле json
import json

with open('/Users/Nikol/Desktop/ВКР/sudact/sudact_157_output_18.json', 'w+') as f:
    json.dump(obj=details_uk, fp=f)

In [111]:
# проверяем список и сохраненный файл на идентичность

with open('/Users/Nikol/Desktop/ВКР/sudact/sudact_157_output_18.json') as f:
    details_uk1 = json.load(f)

details_uk1 == details_uk

True